In [1]:
# 패키지 인스톨
# https://github.com/googleapis/google-api-python-client?tab=readme-ov-file#windows

# !pip install virtualenv
!virtualenv venv
!venv\Scripts\activate.bat
# !venv\Scripts\pip.exe install google-api-python-client

created virtual environment CPython3.10.0.candidate.2-64 in 35266ms
  creator CPython3Windows(dest=C:\Users\tkdgu\OneDrive\문서\Github\idol_info_finder\venv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=C:\Users\tkdgu\AppData\Local\pypa\virtualenv)
    added seed packages: Babel==2.15.0, MarkupSafe==2.1.5, PyYAML==6.0.1, QtPy==2.4.1, Send2Trash==1.8.3, anyio==4.4.0, argon2_cffi==23.1.0, argon2_cffi_bindings==21.2.0, arrow==1.3.0, asttokens==2.4.1, async_lru==2.0.4, attrs==23.2.0, beautifulsoup4==4.12.3, bleach==6.1.0, cachetools==5.4.0, certifi==2024.7.4, cffi==1.16.0, charset_normalizer==3.3.2, colorama==0.4.6, comm==0.2.2, debugpy==1.8.2, decorator==5.1.1, defusedxml==0.7.1, entrypoints==0.4, exceptiongroup==1.2.2, executing==2.0.1, fastjsonschema==2.20.0, fqdn==1.5.1, google_api_core==2.19.1, google_api_python_client==2.137.0, google_auth==2.32.0, google_auth_httplib2==0.2.0, go

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()  # .env 파일에서 환경 변수를 불러옵니다.

# api_key = os.getenv("OPENAI_API_KEY")
# print(f"API Key: {api_key}")
google_api_key = os.getenv("GOOGLE_API_KEY")
search_engine_key = os.getenv("GOOGLE_SEARCH_ENGINE")

In [21]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#
# Copyright 2014 Google Inc. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Simple command-line example for Custom Search.

Command-line application that does a search.
"""

# __author__ = "jcgregorio@google.com (Joe Gregorio)"

import pprint

from googleapiclient.discovery import build


# def main():
# Build a service object for interacting with the API. Visit
# the Google APIs Console <http://code.google.com/apis/console>
# to get an API key for your own application.
service = build(
"customsearch", "v1", developerKey=google_api_key # init로 생성
)

query = 'yungblad' #입력
siteDomain = "https://namu.wiki/w/" #입력 / default
# siteDomain = "https://ko.wikipedia.org/"

res = (
service.cse()
.list(
    q=query,
    cx=search_engine_key, #init로 생성
    siteSearch=siteDomain,
    siteSearchFilter='i'
    
)
.execute()
)
pprint.pprint(res)


# if __name__ == "__main__":
#     main()

{'context': {'title': 'sanghyoon'},
 'kind': 'customsearch#search',
 'queries': {'request': [{'count': 10,
                          'cx': '70fd894c0e18248de',
                          'inputEncoding': 'utf8',
                          'outputEncoding': 'utf8',
                          'safe': 'off',
                          'searchTerms': 'yungblad',
                          'siteSearch': 'https://namu.wiki/w/',
                          'siteSearchFilter': 'i',
                          'startIndex': 1,
                          'title': 'Google Custom Search - yungblad'}]},
 'searchInformation': {'formattedSearchTime': '0.29',
                       'formattedTotalResults': '0',
                       'searchTime': 0.287064,
                       'totalResults': '0'},
 'spelling': {'correctedQuery': 'yungblud site:https://namu.wiki/w/',
              'htmlCorrectedQuery': '<b><i>yungblud</i></b> '
                                    'site:https://namu.wiki/w/'},
 'url': {'templ

In [ ]:
TODO
첫 페이지만 검색해서 가져오면 될지?
# 아니라면 wiki를 가져올 수 있는 API 패턴을 만들 수 있는지?


In [17]:
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [113]:
def get_url(query, siteDomain = "https://namu.wiki/w/"):
    service = build(
    "customsearch", "v1", developerKey=google_api_key # init로 생성
    )

    # query = 'yungblad' #입력
    # siteDomain = "https://namu.wiki/w/" #입력 / default
    # siteDomain = "https://ko.wikipedia.org/"
    
    res = (
    service.cse()
    .list(
        q=query,
        cx=search_engine_key, #init로 생성
        siteSearch=siteDomain,
        siteSearchFilter='i',
        cr = 'countryKR' 
        
    )
    .execute()
    )
    
    # if verbose == True:
    #     pprint.pprint(res)
    
    if int(res['searchInformation']['totalResults']) == 0:
        # Regular expression to capture everything before " site"
        pattern = r'^(.*?) site'

        try:
            text = res['spelling']['correctedQuery']
            
            # Using re.search to find the match
            match = re.search(pattern, text)

            # Extracting and printing the result
            if match:
                adj_query = match.group(1)
                
                res = (
                service.cse()
                .list(
                    q=adj_query,
                    cx=search_engine_key, #init로 생성
                    siteSearch=siteDomain,
                    siteSearchFilter='i',
                    cr = 'countryKR' 
                    
                )
                .execute()
                )
                
        except:
            print("No match found")
            
    url = res['items'][0]['formattedUrl']
    
    return (url, res)

In [114]:
(samp, f) = get_url('fromis_9')

In [115]:
samp

'https://en.namu.wiki/w/fromis_9%20THE100%20아이돌메이드콘서트'

In [126]:
f['items'][5]#['displayLink']

{'kind': 'customsearch#result',
 'title': '박지원(프로미스나인) - 나무위키',
 'htmlTitle': '박지원(프로미스나인) - 나무위키',
 'link': 'https://namu.wiki/w/%EB%B0%95%EC%A7%80%EC%9B%90(%ED%94%84%EB%A1%9C%EB%AF%B8%EC%8A%A4%EB%82%98%EC%9D%B8)',
 'displayLink': 'namu.wiki',
 'snippet': "Jun 24, 2024 ... 11. [ 리얼리티 ]. 프로미스의 방 로고.png. fromis_9 THE100 아이돌메이드콘서트 로고. ... 그리고 V LIVE의 리얼리티 프로였던 'fromis_9 The 100'에서\xa0...",
 'htmlSnippet': 'Jun 24, 2024 <b>...</b> 11. [ 리얼리티 ]. 프로미스의 방 로고.png. <b>fromis_9</b> THE100 아이돌메이드콘서트 로고. ... 그리고 V LIVE의 리얼리티 프로였던 &#39;<b>fromis_9</b> The 100&#39;에서&nbsp;...',
 'formattedUrl': 'https://namu.wiki/w/박지원(프로미스나인)',
 'htmlFormattedUrl': 'https://namu.wiki/w/박지원(프로미스나인)',
 'pagemap': {'cse_thumbnail': [{'src': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSdY_7MXKHwcl5Fxq4AiGOX_VJVhRAtPyJw5AQmOClnlWv2HGxdh09wYnU&s',
    'width': '201',
    'height': '251'}],
  'metatags': [{'application-name': '나무위키',
    'msapplication-starturl': '/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EB%8C

In [155]:
from get_namu_url import getNamuUrl

inst1 = getNamuUrl(google_api_key, search_engine_key)

In [156]:
inst1.get_url("프로미스나인")

NameError: name 'self' is not defined